# Informe de mantenimiento de equipos

## En este informe se analizarán los datos de mantenimiento de los equipos de un centro de estética. Se cuenta con tres archivos CSV que contienen información sobre los tratamientos realizados, los datos de los equipos y los handpieces utilizados.

Importar librerías necesarias y cargar los datos

In [143]:
import pandas as pd
import re
from datetime import datetime, timedelta
import numpy as np

In [144]:
handpiece_df = pd.read_csv('data_311024/handpieces.csv')
drv_df = pd.read_csv('data_311024/datos_drv_it.csv')
treatments_df = pd.read_csv('data_311024/treatments.csv')
treatments_id_df = pd.read_csv('data_311024/treatments_id.csv')

# Análisis de handpieces

In [145]:
# handpiece_df.head()

In [146]:
# Dividir entre 3 la columna "pulse_count" para tener el número real de disparos en pulso. Sobreescribe el dataframe original y la columna "pulse_count", redondealo a 0 decimales y conviértelo a entero
handpiece_df['pulse_count'] = (handpiece_df['pulse_count'] / 3).round(0).astype(int)

# Quito las columnas que no me interesan como las de "energia"
handpiece_df.drop(columns=['pulse_energy', 'burst_energy', 'activation_energy', 'modulation_energy', 'rn'], inplace=True)

# Creo una columna de "vida útil" con 4m de vida para las mangueras que terminan en "1%" y de 2m para las que terminan en "2%" que son de depilación y le resto los disparos por tipo: pulso, ráfaga, etc. según su multiplicador
handpiece_df['vida_util_restante'] = handpiece_df.apply(
    lambda row: 4000000 - (row['pulse_count'] * 20) - (row['burst_count'] * 1) - (row['activation_count'] + row['modulation_count'])
    if re.search(r'1\d$', str(row['handpiece_id'])) else
    2000000 - (row['pulse_count'] * 10) - (row['burst_count'] * 1) - ((row['activation_count'] + row['modulation_count']) * 0.5)
    if re.search(r'2\d$', str(row['handpiece_id'])) else str('Manguera_no_identificada'),
    axis=1
)

# Creo una columna nueva inversa a la anterior que calcula los disparos totales reales en base a la fórmula de disparos (200k en pulso, 2m en ráfaga y 4m en activación/modulación)
handpiece_df['disparos_totales_reales'] = handpiece_df.apply(
    lambda row: (row['pulse_count'] * 20) + (row['burst_count'] * 1) + (row['activation_count'] + row['modulation_count'])
    if re.search(r'1\d$', str(row['handpiece_id'])) else
    (row['pulse_count'] * 10) + (row['burst_count'] * 1) + ((row['activation_count'] + row['modulation_count']) * 0.5)
    if re.search(r'2\d$', str(row['handpiece_id'])) else str('Manguera_no_identificada'),
    axis=1
)

# Los datos dentro de la columna "handpiece_id" aparecen como "a3002020" y me gustaría asignarles un nombre que se entienda entonces: todos los que terminen en "2%" (siendo % cualquier número, como en SQL) los llamaré "F.F." y los que terminen en "1%" los llamaré "M.F."
handpiece_df['handpiece_id'] = handpiece_df['handpiece_id'].apply(
    lambda x: 'F.F.' if re.search(r'2\d$', str(x)) else 'M.F.' if re.search(r'1\d$', str(x)) else 'Manguera_no_identificada'
)

# Le cambio el nombre a la columna "use_count" para llamarla "disparos_totales_brutos"
handpiece_df.rename(columns={'use_count': 'disparos_totales_brutos'}, inplace=True)

# Le cambio el nombre a "created_at" y le asigno "ultima_fecha_uso" y reorganizo las columnas para que estén en el siguiente orden: "ultima_fecha_uso", "din", "serial_number", "prod_date", "disparos_totales_brutos", "disparos_totales_reales", "vida_util_restante", "pulse_count", "burst_count", "activation_count", "modulation_count" 
handpiece_df.rename(columns={'created_at': 'ultima_fecha_uso'}, inplace=True)
handpiece_df = handpiece_df[['ultima_fecha_uso', 'din', 'handpiece_id', 'serial_number', 'prod_date', 'disparos_totales_brutos', 'disparos_totales_reales', 'vida_util_restante', 'pulse_count', 'burst_count', 'activation_count', 'modulation_count']]

# handpiece_df.head()

## Sacar informe intermedio de mangueras gastadas con datos de clientes

In [147]:
# drv_df.head()

In [148]:
# Creo un nuevo dataframe usando "handpiece_df" y "drv_df" para sacar un informe de mangueras gastadas con datos de clientes, este nuevo df es idéntifo a "handpiece_df" pero con las columna "cliente" (client_name), "zona" (area) y "comercial" (salesperson) que se obtiene de "drv_df" que se colocan justo después de "din"  
handpiece_client_df = handpiece_df.merge(drv_df[['din', 'cliente', 'area', 'comercial', 'tipo_pago', 'fecha_corte', 'usado']], how='left', on='din')

# Reorganiza handpiece_client_df y coloca las columnas "client_name", "area" y "salesperson" justo después de "din"
handpiece_client_df = handpiece_client_df[['ultima_fecha_uso', 'din', 'cliente', 'fecha_corte', 'area', 'comercial', 'tipo_pago', 'handpiece_id', 'serial_number', 'prod_date', 'disparos_totales_brutos', 'disparos_totales_reales', 'vida_util_restante', 'pulse_count', 'burst_count', 'activation_count', 'modulation_count']]

# handpiece_client_df.head()

Guardar los datos en un archivo CSV para el informe intermedio

In [149]:
# handpiece_client_df.to_csv('excel/informes_intermedios/informe_gasto_magueras.csv', index=False)

# Análisis de tratamientos

In [150]:
# treatments_df.head()

In [151]:
# treatments_id_df.head()

In [152]:
# Perform a left join on treatment_id from treatments_id_df and code from treatments_df to get the treatment name, if there is not a treatment name, please assign "Tipo" as "FHOS", "Subprograma" as "FHOS genérico", "PVP" as 60 and "Secuencia" as 1.0

treatments_df = treatments_df.merge(treatments_id_df, how='left', left_on='code', right_on='Treatment_ID')

# Assign default values if there is no treatment name
treatments_df = treatments_df.assign(
    Tipo=treatments_df['Tipo'].fillna('FHOS'),
    Subprograma=treatments_df['Subprograma'].fillna('FHOS genérico'),
    PVP=treatments_df['PVP'].fillna(60),
    Secuencia=treatments_df['Secuencia'].fillna(1.0)
)

# Adjust "use_count" if "Tipo_disparo" is "pulso" and round up "use_count" to the nearest integer
treatments_df['use_count'] = treatments_df.apply(
    lambda row: np.ceil(row['use_count'] / 3) if row['Tipo_disparo'] == 'pulso' else np.ceil(row['use_count']), axis=1
)

# Replace infinite values with 0
treatments_df['use_count'] = treatments_df['use_count'].replace([np.inf, -np.inf], 0)

# Fill NaN values with 0
treatments_df['use_count'] = treatments_df['use_count'].fillna(0)

# Convert the column to integers
treatments_df['use_count'] = treatments_df['use_count'].astype(int)


In [153]:
treatments_df.head(100)
# treatments_df.to_csv('excel/informes_intermedios/tratamientos_con_nombre.csv', index=False)

,id,din,code,reported_at,started_at,duration,invoice_duration,use_count,use_unit,Treatment_ID,Tipo,Subtipo,Subprograma,Secuencia,PVP,Tipo_disparo
0,3011974,CM-A30-000101,FHO_FA_AAG,2024-10-31 07:10:43+00,2024-10-31 07:10:43+00,NaN,NaN,5929,shot,FHO_FA_AAG,FHOS,FHOS 15',FHOS Antiage,1.0,60.0,activacion
1,3011971,CM-A37-000034,FHO_FC_FOX,2024-10-31 06:00:14+00,2024-10-31 06:00:14+00,235000.0,240000.0,3744,shot,NaN,FHOS,NaN,FHOS genérico,1.0,60.0,NaN
2,3011970,CM-A37-000034,FHO_FA_FOX,2024-10-31 05:56:00+00,2024-10-31 05:56:00+00,225000.0,240000.0,3026,shot,FHO_FA_FOX,FHOS,FHOS PREMED,FHOTOX,1.0,60.0,activacion
3,3011883,CM-A37-000028,FHO_FS_SG2,2024-10-30 20:00:36+00,2024-10-30 20:00:36+00,303000.0,360000.0,412,shot,FHO_FS_SG2,FHOS,FHOS INTENSE,FHOS Flacidez,2.0,60.0,estimulacion
4,3011876,CM-A37-000028,FHO_FS_SG2,2024-10-30 19:53:25+00,2024-10-30 19:53:25+00,363000.0,420000.0,419,shot,FHO_FS_SG2,FHOS,FHOS INTENSE,FHOS Flacidez,2.0,60.0,estimulacion
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,3011640,CM-A30-000082,DEP_PU,2024-10-30 18:32:29+00,2024-10-30 18:32:29+00,37000.0,60000.0,7,shot,DEP_PU,DEPILACION,DEPILACION,Depilación en pulso,1.0,30.0,pulso
96,3011639,CM-A30-000041,FHO_FA_DCH,2024-10-30 18:32:24+00,2024-10-30 18:32:24+00,807000.0,840000.0,8271,shot,FHO_FA_DCH,FHOS,FHOS INTENSE,FHOS Doble Mentón,1.0,60.0,activacion
97,3011638,CM-A30-000085,FHO_FS_WR1,2024-10-30 18:32:19+00,2024-10-30 18:32:19+00,372000.0,420000.0,499,shot,FHO_FS_WR1,FHOS,FHOS INTENSE,FHOS Arrugas,2.0,60.0,estimulacion
98,3011637,CM-A47-000021,FHO_FS_AAG,2024-10-30 18:32:14+00,2024-10-30 18:32:14+00,645000.0,660000.0,4,shot,FHO_FS_AAG,FHOS,FHOS 15',FHOS Antiage,2.0,60.0,estimulacion


# Análisis de un equipo en particular


In [154]:
# Usando treatments_df y handpiece_df, se puede analizar el uso de un equipo en particular. En este caso, se analizará el equipo con din "CM-A30-000241" que es un buen ejemplo por tener varios handpieces asociados y múltiples tratamientos realizados. Creo la variable DIN y filtro ambos dataframes por ese din.
din = 'CM-A30-000227'

# Filtro handpiece_df por el din, ordernar por "ultima_fecha_uso" y reseteo el índice, con la fecha más cercana a hoy en la parte superior
filtered_handpiece_df = handpiece_client_df.loc[handpiece_client_df['din'] == din].sort_values(by='ultima_fecha_uso', ascending=False).reset_index(drop=True)

# Filtro treatments_df por el din y reseteo el índice, con la fecha más cercana a hoy en la parte superior
filtered_treatments_df = treatments_df.loc[treatments_df['din'] == din].sort_values(by='reported_at', ascending=False).reset_index(drop=True)

In [155]:
# Hacemos un filtro por fecha de corte para filtrar los tratamientos realizados desde la fecha de corte hasta hoy, creo la variable "fecha_corte", en el caso de que la fecha sea anterior al 1 de enero de 2023, dejo esta fecha como fecha de corte, empiezo filtrando "filtered_treatments_df" por la fecha de corte y reseteo el índice, usar por defecto la "fecha_corte" de filtered_handpiece_df y si esta no existe o es anterior al 1 de enero de 2023, usar "2023-01-01"
# Ensure 'fecha_corte' column is in datetime format
filtered_handpiece_df['fecha_corte'] = pd.to_datetime(filtered_handpiece_df['fecha_corte'], errors='coerce')

# Get the minimum date from 'fecha_corte' column
min_fecha_corte = filtered_handpiece_df['fecha_corte'].min()

# Set 'fecha_corte' to '2023-01-01' if 'min_fecha_corte' is NaT or earlier than '2023-01-01'
fecha_corte = '2023-01-01' if pd.isna(min_fecha_corte) or min_fecha_corte < pd.Timestamp('2023-01-01') else min_fecha_corte.strftime('%Y-%m-%d')

# Convertimos la columna 'reported_at' a datetime si no lo está
filtered_treatments_df['reported_at'] = pd.to_datetime(filtered_treatments_df['reported_at'])

# Filtramos por la fecha de corte
filtered_treatments_df = filtered_treatments_df[filtered_treatments_df['reported_at'] >= fecha_corte].reset_index(drop=True)

# Convertimos la columna 'reported_at' a datetime si no lo está
filtered_handpiece_df['ultima_fecha_uso'] = pd.to_datetime(filtered_handpiece_df['ultima_fecha_uso'])

# Filtramos por la fecha de corte
filtered_handpiece_df = filtered_handpiece_df[filtered_handpiece_df['ultima_fecha_uso'] >= fecha_corte].reset_index(drop=True)

In [156]:
# filtered_handpiece_df.head()

### Análisis y filtro de la tabla de handpieces


In [157]:
# filtered_treatments_df.head()
# filtered_treatments_df.to_csv('excel/informes_intermedios/tratamientos_intermedios.csv', index=False)

# Crear un campo nuevo que se llame "Estado" y asignarle "Activo" si la "ultima_fecha_uso" es de hace menos de 30 días con respecto a la fecha de hoy e "Inactivo" si es anterior a esa fecha, asimismo elimina los campos "salesperson" y "area" del dataframe "filtered_handpiece_df" y cambia el nombre de "pulse_count" a "disparos_pulso", "burst_count" a "disparos_rafaga", "activation_count" a "disparos_activacion" y "modulation_count" a "disparos_modulacion"
# Ensure 'ultima_fecha_uso' is timezone-naive
filtered_handpiece_df['ultima_fecha_uso'] = pd.to_datetime(filtered_handpiece_df['ultima_fecha_uso']).dt.tz_localize(None)

# Create a new field "Estado"
today = datetime.now()  # Use timezone-naive current datetime
filtered_handpiece_df['estado'] = filtered_handpiece_df['ultima_fecha_uso'].apply(
    lambda x: 'ACTIVO' if (today - x).days < 30 else 'INACTIVO'
)

# Remove the columns "salesperson" and "area"
filtered_handpiece_df.drop(columns=['comercial', 'area'], inplace=True)

# Rename the columns
filtered_handpiece_df.rename(columns={
    'handpiece_id': 'tipo_manipulo',
    'serial_number': 'numero_serie',
    'disparos_totales_reales': 'disparos_reales',
    'pulse_count': 'disparos_pulso',
    'burst_count': 'disparos_rafaga',
    'activation_count': 'disparos_activacion',
    'modulation_count': 'disparos_modulacion'
}, inplace=True)

# Apply multipliers based on 'tipo_manipulo'
# filtered_handpiece_df['disparos_pulso'] = filtered_handpiece_df.apply(
#     lambda row: row['disparos_pulso'] * 20 if row['tipo_manipulo'] == 'M.F.' else row['disparos_pulso'] * 10, axis=1
# )
# filtered_handpiece_df['disparos_rafaga'] = filtered_handpiece_df.apply(
#     lambda row: row['disparos_rafaga'] * 2 if row['tipo_manipulo'] == 'M.F.' else row['disparos_rafaga'] * 1, axis=1
# )
# filtered_handpiece_df['disparos_activacion'] = filtered_handpiece_df.apply(
#     lambda row: row['disparos_activacion'] * 1 if row['tipo_manipulo'] == 'M.F.' else row['disparos_activacion'] * 0.5, axis=1
# )
# filtered_handpiece_df['disparos_modulacion'] = filtered_handpiece_df.apply(
#     lambda row: row['disparos_modulacion'] * 1 if row['tipo_manipulo'] == 'M.F.' else row['disparos_modulacion'] * 0.5, axis=1
# )

# Reorder the columns
filtered_handpiece_df = filtered_handpiece_df[[
    'ultima_fecha_uso', 'estado', 'din', 'cliente', 'tipo_manipulo', 'numero_serie', 'disparos_pulso', 'disparos_rafaga', 'disparos_activacion', 'disparos_modulacion'
]]

# , 'disparos_reales'
filtered_handpiece_df.head(20)

# SKINIC: ((pulse/3)*20) + (burst*1) + (activation + modulation)
# 4.000.000/200.000 = 20

,ultima_fecha_uso,estado,din,cliente,tipo_manipulo,numero_serie,disparos_pulso,disparos_rafaga,disparos_activacion,disparos_modulacion
0,2024-10-23 07:38:21,ACTIVO,CM-A30-000227,"NOVARED MC, S.L.",F.F.,1037,194,3,0,0
1,2024-10-23 07:38:21,ACTIVO,CM-A30-000227,"NOVARED MC, S.L.",M.F.,826,646904,362396,2239798,405929
2,2024-10-18 08:06:56,ACTIVO,CM-A30-000227,"NOVARED MC, S.L.",F.F.,1294,0,0,0,0
3,2024-10-18 07:56:15,ACTIVO,CM-A30-000227,"NOVARED MC, S.L.",F.F.,120030,13,0,0,0
4,2024-10-17 10:49:37,ACTIVO,CM-A30-000227,"NOVARED MC, S.L.",M.F.,1504,487,10156,98178,4405
5,2024-10-17 10:31:15,ACTIVO,CM-A30-000227,"NOVARED MC, S.L.",M.F.,1457,86916,80096,68,165200
6,2024-10-17 10:27:30,ACTIVO,CM-A30-000227,"NOVARED MC, S.L.",M.F.,1138,1392,10065,60363,28323
7,2024-10-17 10:17:40,ACTIVO,CM-A30-000227,"NOVARED MC, S.L.",F.F.,1435,20,7222,0,0
8,2024-10-14 10:12:30,ACTIVO,CM-A30-000227,"NOVARED MC, S.L.",F.F.,1288,30,777892,0,0
9,2024-10-14 09:55:06,ACTIVO,CM-A30-000227,"NOVARED MC, S.L.",F.F.,1489,83,2143716,4194806784,4294967295


### Análisis y filtro de la tabla de tratamientos

In [158]:
filtered_treatments_df.head(100)

,id,din,code,reported_at,started_at,duration,invoice_duration,use_count,use_unit,Treatment_ID,Tipo,Subtipo,Subprograma,Secuencia,PVP,Tipo_disparo
0,2984923,CM-A30-000227,DEP_PU,2024-10-23 07:39:13+00:00,2024-10-23 07:39:13+00,21000.0,60000.0,1,shot,DEP_PU,DEPILACION,DEPILACION,Depilación en pulso,1.0,30.0,pulso
1,2984918,CM-A30-000227,DEP_PU,2024-10-23 07:38:28+00:00,2024-10-23 07:38:28+00,42000.0,60000.0,1,shot,DEP_PU,DEPILACION,DEPILACION,Depilación en pulso,1.0,30.0,pulso
2,2967504,CM-A30-000227,DEP_PU,2024-10-18 08:16:19+00:00,2024-10-18 08:16:19+00,116000.0,120000.0,2,shot,DEP_PU,DEPILACION,DEPILACION,Depilación en pulso,1.0,30.0,pulso
3,2967465,CM-A30-000227,DEP_PU,2024-10-18 08:12:34+00:00,2024-10-18 08:12:34+00,61000.0,120000.0,1,shot,DEP_PU,DEPILACION,DEPILACION,Depilación en pulso,1.0,30.0,pulso
4,2967462,CM-A30-000227,DEP_PU,2024-10-18 08:12:08+00:00,2024-10-18 08:12:08+00,23000.0,60000.0,1,shot,DEP_PU,DEPILACION,DEPILACION,Depilación en pulso,1.0,30.0,pulso
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2937830,CM-A30-000227,DEP_PU,2024-10-09 14:52:16+00:00,2024-10-09 11:44:55+00,85000.0,120000.0,0,NaN,DEP_PU,DEPILACION,DEPILACION,Depilación en pulso,1.0,30.0,pulso
96,2937828,CM-A30-000227,DEP_PU,2024-10-09 14:51:57+00:00,2024-10-09 10:26:37+00,4000.0,0.0,0,NaN,DEP_PU,DEPILACION,DEPILACION,Depilación en pulso,1.0,30.0,pulso
97,2892136,CM-A30-000227,FHO_BS_RES,2024-09-26 10:36:29+00:00,2024-09-26 10:36:29+00,391000.0,420000.0,1411,shot,FHO_BS_RES,FHOS,FHOS 15',FHOS Remodelación,2.0,60.0,estimulacion
98,2892080,CM-A30-000227,FHO_BA_RES,2024-09-26 10:24:47+00:00,2024-09-26 10:24:46+00,491000.0,540000.0,10517,shot,FHO_BA_RES,FHOS,FHOS 15',FHOS Remodelación,1.0,60.0,activacion


In [159]:
# Creo una tabla resumen que sume los tratamientos por "Tipo" y "Cantidad" y la ordeno por "Cantidad" de mayor a menor con una columna adicional siempre y cuando "Secuencia" sea igual a 1.0 en el tratamiento, si es distinto, lo ignora. Asimismo que sume una columna, con la misma condición de "Secuencia" con la suma de PVP
resumen_tratamientos = filtered_treatments_df.loc[filtered_treatments_df['Secuencia'] == 1.0].groupby('Tipo').agg(
    Cantidad=('Tipo', 'count'),
    PVP=('PVP', 'sum'),
).sort_values(by='Cantidad', ascending=False).reset_index()

# Calculate the total "Cantidad" and "PVP"
total_cantidad = resumen_tratamientos['Cantidad'].sum()
total_pvp = resumen_tratamientos['PVP'].sum()

# Add percentage columns
resumen_tratamientos['% de Tratamientos'] = (resumen_tratamientos['Cantidad'] / total_cantidad * 100).round(2)
resumen_tratamientos['% de Ingresos'] = (resumen_tratamientos['PVP'] / total_pvp * 100).round(2)

resumen_tratamientos.head()

,Tipo,Cantidad,PVP,% de Tratamientos,% de Ingresos
0,DEPILACION,1121,33630.0,58.51,41.36
1,FHOS,794,47640.0,41.44,58.59
2,FHOS C.A.,1,45.0,0.05,0.06


In [160]:
resumen_tratamientos.to_csv('excel/informes_intermedios/novared/A30227/resumen_tratamientos.csv', index=False)

In [161]:
# Creo una tabla resumen que sume los tratamientos por "Subprograma" y "Cantidad" y la ordeno por "Cantidad" de mayor a menor con una columna adicional siempre y cuando "Secuencia" sea igual a 1.0 en el tratamiento, si es distinto, lo ignora. Asimismo que sume una columna, con la misma condición de "Secuencia" con la suma de PVP
resumen_subprogramas = filtered_treatments_df.loc[filtered_treatments_df['Secuencia'] == 1.0].groupby('Subprograma').agg(
    Cantidad=('Subprograma', 'count'),
    PVP=('PVP', 'sum'),
).sort_values(by='Cantidad', ascending=False).reset_index()

# Calculate the total "Cantidad" and "PVP"
total_cantidad = resumen_subprogramas['Cantidad'].sum()
total_pvp = resumen_subprogramas['PVP'].sum()

# Add percentage columns
resumen_subprogramas['% de Tratamientos'] = (resumen_subprogramas['Cantidad'] / total_cantidad * 100).round(2)
resumen_subprogramas['% de Ingresos'] = (resumen_subprogramas['PVP'] / total_pvp * 100).round(2)

resumen_subprogramas.head(100)

,Subprograma,Cantidad,PVP,% de Tratamientos,% de Ingresos
0,Depilación en pulso,887,26610.0,46.29,32.72
1,FHOS genérico,356,21360.0,18.58,26.27
2,Depilación en ráfaga,234,7020.0,12.21,8.63
3,FHOS Antiage,95,5700.0,4.96,7.01
4,FHOS Reductor,50,3000.0,2.61,3.69
5,FHOS Flacidez,40,2400.0,2.09,2.95
6,FHOS Reparador,34,2040.0,1.77,2.51
7,FHOS Arrugas,29,1740.0,1.51,2.14
8,FHOS Remodelación,25,1500.0,1.30,1.84
9,FHOS Doble Mentón,23,1380.0,1.20,1.70


In [162]:
# Create a summary table that sums the treatments by "Subprograma" and "Cantidad" and orders by "Cantidad" in descending order, only if "Secuencia" is equal to 1.0
resumen_subprogramas = filtered_treatments_df.loc[filtered_treatments_df['Secuencia'] == 1.0].groupby('Subprograma').agg(
    Cantidad=('Subprograma', 'count'),
    PVP=('PVP', 'sum'),
).sort_values(by='Cantidad', ascending=False).reset_index()

# Calculate the total "Cantidad" and "PVP"
total_cantidad = resumen_subprogramas['Cantidad'].sum()
total_pvp = resumen_subprogramas['PVP'].sum()

# Add percentage columns
resumen_subprogramas['% de Tratamientos'] = (resumen_subprogramas['Cantidad'] / total_cantidad * 100).round(2)
resumen_subprogramas['% de Ingresos'] = (resumen_subprogramas['PVP'] / total_pvp * 100).round(2)

# Create a summary table for "use_count" broken down by "Tipo_disparo"
resumen_disparos = filtered_treatments_df.loc[filtered_treatments_df['Secuencia'] == 1.0].groupby('Tipo_disparo').agg(
    Disparos=('use_count', 'sum')
).reset_index()

# Display the summary tables
print(resumen_subprogramas.head(100))
print(resumen_disparos.head(100))

               Subprograma  Cantidad      PVP  % de Tratamientos  \
0      Depilación en pulso       887  26610.0              46.29   
1            FHOS genérico       356  21360.0              18.58   
2     Depilación en ráfaga       234   7020.0              12.21   
3             FHOS Antiage        95   5700.0               4.96   
4            FHOS Reductor        50   3000.0               2.61   
5            FHOS Flacidez        40   2400.0               2.09   
6           FHOS Reparador        34   2040.0               1.77   
7             FHOS Arrugas        29   1740.0               1.51   
8        FHOS Remodelación        25   1500.0               1.30   
9        FHOS Doble Mentón        23   1380.0               1.20   
10             FHOS Tensor        20   1200.0               1.04   
11         FHOS Piel Grasa        18   1080.0               0.94   
12           FHOS Calmante        16    960.0               0.84   
13     FHOS Imperfecciones        16    960.0   

In [163]:
# Create a summary table for "use_count" broken down by "Tipo_disparo" without filtering by "Secuencia"
resumen_disparos_todos = filtered_treatments_df.groupby('Tipo_disparo').agg(
    Disparos=('use_count', 'sum')
).reset_index()

# Display the summary table
print(resumen_disparos_todos.head())

   Tipo_disparo  Disparos
0    activacion   3575738
1  estimulacion    352966
2    modulacion    405929
3         pulso    158698
4        rafaga    230887


## CSV de tratamientos para la clienta

In [164]:
filtered_treatments_df.head(100)

,id,din,code,reported_at,started_at,duration,invoice_duration,use_count,use_unit,Treatment_ID,Tipo,Subtipo,Subprograma,Secuencia,PVP,Tipo_disparo
0,2984923,CM-A30-000227,DEP_PU,2024-10-23 07:39:13+00:00,2024-10-23 07:39:13+00,21000.0,60000.0,1,shot,DEP_PU,DEPILACION,DEPILACION,Depilación en pulso,1.0,30.0,pulso
1,2984918,CM-A30-000227,DEP_PU,2024-10-23 07:38:28+00:00,2024-10-23 07:38:28+00,42000.0,60000.0,1,shot,DEP_PU,DEPILACION,DEPILACION,Depilación en pulso,1.0,30.0,pulso
2,2967504,CM-A30-000227,DEP_PU,2024-10-18 08:16:19+00:00,2024-10-18 08:16:19+00,116000.0,120000.0,2,shot,DEP_PU,DEPILACION,DEPILACION,Depilación en pulso,1.0,30.0,pulso
3,2967465,CM-A30-000227,DEP_PU,2024-10-18 08:12:34+00:00,2024-10-18 08:12:34+00,61000.0,120000.0,1,shot,DEP_PU,DEPILACION,DEPILACION,Depilación en pulso,1.0,30.0,pulso
4,2967462,CM-A30-000227,DEP_PU,2024-10-18 08:12:08+00:00,2024-10-18 08:12:08+00,23000.0,60000.0,1,shot,DEP_PU,DEPILACION,DEPILACION,Depilación en pulso,1.0,30.0,pulso
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2937830,CM-A30-000227,DEP_PU,2024-10-09 14:52:16+00:00,2024-10-09 11:44:55+00,85000.0,120000.0,0,NaN,DEP_PU,DEPILACION,DEPILACION,Depilación en pulso,1.0,30.0,pulso
96,2937828,CM-A30-000227,DEP_PU,2024-10-09 14:51:57+00:00,2024-10-09 10:26:37+00,4000.0,0.0,0,NaN,DEP_PU,DEPILACION,DEPILACION,Depilación en pulso,1.0,30.0,pulso
97,2892136,CM-A30-000227,FHO_BS_RES,2024-09-26 10:36:29+00:00,2024-09-26 10:36:29+00,391000.0,420000.0,1411,shot,FHO_BS_RES,FHOS,FHOS 15',FHOS Remodelación,2.0,60.0,estimulacion
98,2892080,CM-A30-000227,FHO_BA_RES,2024-09-26 10:24:47+00:00,2024-09-26 10:24:46+00,491000.0,540000.0,10517,shot,FHO_BA_RES,FHOS,FHOS 15',FHOS Remodelación,1.0,60.0,activacion


In [165]:
# Quitar las columnas "reported_at", "duration", "invoice_duration", "use_count", "use_unit", "Treatment_ID", "Tipo", "Subtipo", "Subprograma", "PVP", "Secuencia" y "Tipo_disparo" y crear una nueva columna "duracion (min)" que sea igual a "invoice_duration" / 60000
filtered_treatments_df['duracion'] = filtered_treatments_df['duration'] / 1000

# Rename the columns
filtered_treatments_df.rename(columns={
    'started_at': 'fecha',
    'code': 'codigo_tratamiento'
}, inplace=True)

filtered_treatments_df.drop(columns=[
    'reported_at', 'duration', 'invoice_duration', 'use_count', 'use_unit', 'Treatment_ID', 'Tipo', 'Subtipo', 'Subprograma', 'PVP', 'Secuencia', 'Tipo_disparo'
], inplace=True)



In [166]:
filtered_treatments_df.head(100)

,id,din,codigo_tratamiento,fecha,duracion
0,2984923,CM-A30-000227,DEP_PU,2024-10-23 07:39:13+00,21.0
1,2984918,CM-A30-000227,DEP_PU,2024-10-23 07:38:28+00,42.0
2,2967504,CM-A30-000227,DEP_PU,2024-10-18 08:16:19+00,116.0
3,2967465,CM-A30-000227,DEP_PU,2024-10-18 08:12:34+00,61.0
4,2967462,CM-A30-000227,DEP_PU,2024-10-18 08:12:08+00,23.0
...,...,...,...,...,...
95,2937830,CM-A30-000227,DEP_PU,2024-10-09 11:44:55+00,85.0
96,2937828,CM-A30-000227,DEP_PU,2024-10-09 10:26:37+00,4.0
97,2892136,CM-A30-000227,FHO_BS_RES,2024-09-26 10:36:29+00,391.0
98,2892080,CM-A30-000227,FHO_BA_RES,2024-09-26 10:24:46+00,491.0
